# Chapter 31: Internationalization (i18n)

As applications scale globally, supporting multiple languages becomes essential for reaching diverse audiences. Next.js App Router provides flexible internationalization patterns that go beyond simple translation strings, enabling locale-specific routing, automatic language detection, and SEO-optimized multi-language architectures. Implementing i18n requires careful consideration of routing strategies, content management, and performance optimization across different character sets and text directions.

By the end of this chapter, you'll master configuring locale-based routing with middleware, implementing translation dictionaries with type safety, handling RTL (Right-to-Left) languages, formatting dates and numbers for specific locales, managing localized content with CMS integration, and optimizing SEO with hreflang attributes and metadata.

## 31.1 i18n Setup with next-intl

Configure a type-safe internationalization system for the App Router using next-intl, the recommended approach for modern Next.js applications.

### Configuration and Middleware

```typescript
// middleware.ts
import createMiddleware from 'next-intl/middleware';
import { NextRequest } from 'next/server';

export default async function middleware(request: NextRequest) {
  // Default locale when no match is found
  const defaultLocale = 'en';
  
  const handleI18nRouting = createMiddleware({
    // A list of all locales that are supported
    locales: ['en', 'es', 'fr', 'de', 'ar', 'ja'],
    
    // Used when no locale matches
    defaultLocale,
    
    // Locale detection strategy
    localeDetection: true,
    
    // Prefix strategy: 'always' | 'as-needed' | 'never'
    // 'as-needed' hides prefix for default locale
    localePrefix: 'as-needed',
  });

  const response = handleI18nRouting(request);
  
  // Add security headers or additional middleware logic
  response.headers.set('X-Locale', request.headers.get('X-Locale') || defaultLocale);
  
  return response;
}

export const config = {
  // Match only internationalized pathnames
  matcher: ['/((?!api|_next|.*\\..*).*)']
};
```

```typescript
// next.config.js
const withNextIntl = require('next-intl/plugin')();

/** @type {import('next').NextConfig} */
const nextConfig = {
  // i18n settings are handled by next-intl plugin
};

module.exports = withNextIntl(nextConfig);
```

```typescript
// i18n.ts
import { getRequestConfig } from 'next-intl/server';
import { notFound } from 'next/navigation';

// Can be imported from a shared config
const locales = ['en', 'es', 'fr', 'de', 'ar', 'ja'];

export default getRequestConfig(async ({ locale }) => {
  // Validate that the incoming `locale` parameter is valid
  if (!locales.includes(locale as any)) notFound();

  return {
    messages: (await import(`./messages/${locale}.json`)).default,
    timeZone: 'UTC',
    now: new Date(),
    // Formats for dates, numbers, and lists
    formats: {
      dateTime: {
        short: {
          day: 'numeric',
          month: 'short',
          year: 'numeric',
        },
        long: {
          day: 'numeric',
          month: 'long',
          year: 'numeric',
          hour: 'numeric',
          minute: 'numeric',
        },
      },
      number: {
        precise: {
          maximumFractionDigits: 5,
        },
        currency: {
          style: 'currency',
          currency: 'USD',
        },
      },
    },
  };
});
```

### Type-Safe Translations

```typescript
// types/i18n.ts
import { formats } from '@/i18n';

type Messages = typeof import('@/messages/en.json');
declare module 'next-intl' {
  interface AppConfig {
    Messages: Messages;
    Formats: typeof formats;
  }
}

// messages/en.json
{
  "metadata": {
    "title": "Next.js Mastery Guide",
    "description": "Master modern web development with Next.js"
  },
  "navigation": {
    "home": "Home",
    "courses": "Courses",
    "dashboard": "Dashboard",
    "signIn": "Sign In",
    "signOut": "Sign Out"
  },
  "hero": {
    "title": "Master Next.js Development",
    "subtitle": "From fundamentals to advanced patterns",
    "cta": "Start Learning",
    "stats": {
      "students": "{count} students enrolled",
      "rating": "{rating} out of 5 stars"
    }
  },
  "error": {
    "notFound": "Page not found",
    "generic": "Something went wrong",
    "retry": "Try again"
  }
}
```

## 31.2 Locale Detection and Routing

Implement intelligent locale detection with user preferences and automatic redirection.

### Locale Switching Component

```typescript
// components/locale-switcher.tsx
'use client';

import { useLocale, useTranslations } from 'next-intl';
import { useRouter, usePathname } from '@/navigation';
import { useTransition } from 'react';

const locales = [
  { code: 'en', label: 'English', flag: '🇺🇸' },
  { code: 'es', label: 'Español', flag: '🇪🇸' },
  { code: 'fr', label: 'Français', flag: '🇫🇷' },
  { code: 'de', label: 'Deutsch', flag: '🇩🇪' },
  { code: 'ar', label: 'العربية', flag: '🇸🇦' },
  { code: 'ja', label: '日本語', flag: '🇯🇵' },
];

export function LocaleSwitcher() {
  const locale = useLocale();
  const router = useRouter();
  const pathname = usePathname();
  const [isPending, startTransition] = useTransition();

  function onSelectChange(nextLocale: string) {
    startTransition(() => {
      router.replace(pathname, { locale: nextLocale });
    });
  }

  return (
    <div className="relative group">
      <button 
        className="flex items-center gap-2 px-3 py-2 rounded-lg border hover:bg-gray-50"
        disabled={isPending}
      >
        <span>{locales.find(l => l.code === locale)?.flag}</span>
        <span className="text-sm font-medium">
          {locales.find(l => l.code === locale)?.label}
        </span>
      </button>
      
      <div className="absolute right-0 mt-2 w-48 bg-white rounded-lg shadow-lg border opacity-0 invisible group-hover:opacity-100 group-hover:visible transition-all">
        {locales.map((l) => (
          <button
            key={l.code}
            onClick={() => onSelectChange(l.code)}
            className={`w-full flex items-center gap-3 px-4 py-2 text-left hover:bg-gray-50 first:rounded-t-lg last:rounded-b-lg ${
              locale === l.code ? 'bg-blue-50 text-blue-600' : ''
            }`}
          >
            <span>{l.flag}</span>
            <span className="text-sm">{l.label}</span>
            {locale === l.code && (
              <CheckIcon className="w-4 h-4 ml-auto" />
            )}
          </button>
        ))}
      </div>
    </div>
  );
}
```

```typescript
// navigation.ts
import { createSharedPathnamesNavigation } from 'next-intl/navigation';
import { locales, localePrefix } from './config';

export const { Link, redirect, usePathname, useRouter } = 
  createSharedPathnamesNavigation({ locales, localePrefix });
```

### Server-Side Locale Handling

```typescript
// app/[locale]/layout.tsx
import { NextIntlClientProvider } from 'next-intl';
import { getMessages, getTranslations } from 'next-intl/server';
import { notFound } from 'next/navigation';
import { ReactNode } from 'react';
import { locales } from '@/config';

export function generateStaticParams() {
  return locales.map((locale) => ({ locale }));
}

export async function generateMetadata({ params: { locale } }: { params: { locale: string } }) {
  const t = await getTranslations({ locale, namespace: 'metadata' });
  
  return {
    title: t('title'),
    description: t('description'),
    alternates: {
      canonical: `/${locale}`,
      languages: {
        'en-US': '/en',
        'es-ES': '/es',
        'fr-FR': '/fr',
        'ar-SA': '/ar',
      },
    },
  };
}

export default async function LocaleLayout({
  children,
  params: { locale }
}: {
  children: ReactNode;
  params: { locale: string };
}) {
  // Validate locale
  if (!locales.includes(locale as any)) notFound();

  const messages = await getMessages();
  
  // Determine text direction
  const isRTL = locale === 'ar' || locale === 'he';
  
  return (
    <html lang={locale} dir={isRTL ? 'rtl' : 'ltr'}>
      <body className={isRTL ? 'font-arabic' : ''}>
        <NextIntlClientProvider 
          messages={messages} 
          locale={locale}
          timeZone="UTC"
        >
          {children}
        </NextIntlClientProvider>
      </body>
    </html>
  );
}
```

## 31.3 Localized Content and Data Fetching

Fetch locale-specific content from databases or CMS platforms.

### Database Queries with Locale

```typescript
// lib/db/queries.ts
import { cache } from 'react';
import { db } from './client';

export const getLocalizedContent = cache(async (slug: string, locale: string) => {
  // Try specific locale first
  const content = await db.content.findUnique({
    where: {
      slug_locale: {
        slug,
        locale,
      },
    },
  });

  // Fall back to default locale if not found
  if (!content && locale !== 'en') {
    return await db.content.findUnique({
      where: {
        slug_locale: {
          slug,
          locale: 'en',
        },
      },
    });
  }

  return content;
});

// app/[locale]/blog/[slug]/page.tsx
import { getLocalizedContent } from '@/lib/db/queries';
import { notFound } from 'next/navigation';
import { useTranslations } from 'next-intl';

export default async function BlogPost({ 
  params: { locale, slug } 
}: { 
  params: { locale: string; slug: string } 
}) {
  const post = await getLocalizedContent(slug, locale);
  
  if (!post) notFound();
  
  const t = useTranslations('blog');

  return (
    <article className="max-w-3xl mx-auto py-12">
      <header className="mb-8">
        <h1 className="text-4xl font-bold mb-4">{post.title}</h1>
        <time className="text-gray-500">
          {new Date(post.publishedAt).toLocaleDateString(locale, {
            year: 'numeric',
            month: 'long',
            day: 'numeric',
          })}
        </time>
      </header>
      
      <div 
        className="prose prose-lg max-w-none"
        dangerouslySetInnerHTML={{ __html: post.content }} 
      />
      
      <div className="mt-12 pt-8 border-t">
        <p className="text-sm text-gray-500">
          {t('lastUpdated')}: {post.updatedAt.toLocaleDateString(locale)}
        </p>
      </div>
    </article>
  );
}
```

### Localization with Sanity CMS

```typescript
// lib/sanity/queries.ts
import { groq } from 'next-sanity';
import { client } from './client';

export const getLocalizedPost = async (slug: string, locale: string) => {
  const query = groq`*[_type == "post" && slug.current == $slug][0]{
    _id,
    title,
    "slug": slug.current,
    "content": content[_key == $locale][0].value,
    "excerpt": excerpt[_key == $locale][0].value,
    publishedAt,
    "author": author->{name, image},
    "localizedImage": mainImage{
      ...,
      "alt": alt[_key == $locale][0].value
    }
  }`;

  return await client.fetch(query, { slug, locale });
};

// Sanity schema structure for localized content
const postSchema = {
  name: 'post',
  type: 'document',
  fields: [
    {
      name: 'title',
      type: 'localizedString',
      // Custom type that stores object: { en: 'Hello', es: 'Hola', ... }
    },
    {
      name: 'content',
      type: 'localizedText',
    },
    {
      name: 'slug',
      type: 'slug',
      options: {
        source: 'title.en', // Use default locale for slug generation
        maxLength: 96,
      },
    },
  ],
};
```

## 31.4 Date, Time, and Number Formatting

Implement locale-aware formatting using the Intl API and next-intl hooks.

### Formatting Components

```typescript
// components/formatted-date.tsx
'use client';

import { useFormatter, useTimeZone, useNow } from 'next-intl';

interface FormattedDateProps {
  date: Date | number;
  format?: 'short' | 'long' | 'relative';
}

export function FormattedDate({ date, format = 'long' }: FormattedDateProps) {
  const formatDate = useFormatter();
  const timeZone = useTimeZone();
  
  if (format === 'relative') {
    return <time dateTime={new Date(date).toISOString()}>
      {formatDate.relativeTime(date)}
    </time>;
  }

  const formatOptions = {
    short: { dateStyle: 'short' },
    long: { dateStyle: 'long', timeStyle: 'short' },
  }[format];

  return (
    <time dateTime={new Date(date).toISOString()}>
      {formatDate.dateTime(date, { ...formatOptions, timeZone })}
    </time>
  );
}

// components/currency-display.tsx
'use client';

import { useFormatter } from 'next-intl';

interface CurrencyDisplayProps {
  amount: number;
  currency?: string;
  maximumFractionDigits?: number;
}

export function CurrencyDisplay({ 
  amount, 
  currency = 'USD',
  maximumFractionDigits = 2 
}: CurrencyDisplayProps) {
  const formatNumber = useFormatter();
  
  return (
    <span>
      {formatNumber.number(amount, {
        style: 'currency',
        currency,
        maximumFractionDigits,
      })}
    </span>
  );
}

// Usage with locale-aware formatting
// app/[locale]/products/[id]/page.tsx
import { getTranslations } from 'next-intl/server';

export default async function ProductPage({ 
  params: { locale, id } 
}: { 
  params: { locale: string; id: string } 
}) {
  const t = await getTranslations('product');
  const product = await getProduct(id);
  
  // Format price based on locale
  const price = new Intl.NumberFormat(locale, {
    style: 'currency',
    currency: product.currency,
  }).format(product.price);

  return (
    <div>
      <h1>{product.name}</h1>
      <p className="text-2xl font-bold">{price}</p>
      <p>
        {t('stock', { count: product.stock })} // "5 items in stock" or "1 item in stock"
      </p>
    </div>
  );
}
```

### Pluralization and Rich Text

```json
// messages/en.json
{
  "cart": {
    "items": "{count, plural, =0 {No items} one {# item} other {# items}} in cart",
    "discount": "Save {discount, number, percent}",
    "checkout": "Proceed to checkout",
    "richText": "Welcome, <b>{name}</b>! You have <link>{count} new messages</link>."
  }
}
```

```typescript
// components/cart-summary.tsx
'use client';

import { useTranslations } from 'next-intl';
import Link from 'next/link';

export function CartSummary({ count, discount }: { count: number; discount: number }) {
  const t = useTranslations('cart');
  
  return (
    <div className="p-4 bg-gray-50 rounded-lg">
      <p className="font-medium">
        {t('items', { count })}
      </p>
      
      {discount > 0 && (
        <p className="text-green-600">
          {t('discount', { discount: discount / 100 })}
        </p>
      )}
      
      {/* Rich text with components */}
      <p>
        {t.rich('richText', {
          name: 'John',
          count: 5,
          b: (chunks) => <strong>{chunks}</strong>,
          link: (chunks) => <Link href="/messages" className="text-blue-600 underline">{chunks}</Link>,
        })}
      </p>
    </div>
  );
}
```

## 31.5 Right-to-Left (RTL) Language Support

Implement comprehensive RTL support for Arabic, Hebrew, and other RTL languages.

### Tailwind Configuration for RTL

```typescript
// tailwind.config.ts
import type { Config } from 'tailwindcss';

const config: Config = {
  content: [
    './pages/**/*.{js,ts,jsx,tsx,mdx}',
    './components/**/*.{js,ts,jsx,tsx,mdx}',
    './app/**/*.{js,ts,jsx,tsx,mdx}',
  ],
  theme: {
    extend: {
      fontFamily: {
        sans: ['Inter', 'sans-serif'],
        arabic: ['Noto Sans Arabic', 'sans-serif'],
      },
    },
  },
  plugins: [
    require('tailwindcss-rtl'), // or use logical properties
  ],
  // Enable logical properties for automatic LTR/RTL handling
  corePlugins: {
    textAlign: true,
  },
};
```

```typescript
// components/rtl-layout.tsx
'use client';

import { useLocale } from 'next-intl';
import { useEffect } from 'react';

export function RTLLayout({ children }: { children: React.ReactNode }) {
  const locale = useLocale();
  const isRTL = locale === 'ar' || locale === 'he' || locale === 'ur';
  
  useEffect(() => {
    // Update document direction
    document.documentElement.dir = isRTL ? 'rtl' : 'ltr';
    
    // Load RTL-specific styles if needed
    if (isRTL) {
      import('@/styles/rtl-overrides.css');
    }
  }, [isRTL]);
  
  return (
    <div className={isRTL ? 'font-arabic' : ''}>
      {children}
    </div>
  );
}

// Using logical properties instead of directional ones
// components/navigation.tsx
export function Navigation() {
  return (
    <nav className="flex items-center justify-between px-6 py-4">
      {/* Logical margin: ms-4 = margin-inline-start (left in LTR, right in RTL) */}
      <div className="flex items-center gap-4">
        <Logo className="h-8" />
        <span className="ms-4 text-xl font-bold">Mastery Guide</span>
      </div>
      
      {/* Logical border: border-e = border-inline-end */}
      <div className="flex items-center gap-4 border-e pe-4">
        <LocaleSwitcher />
      </div>
    </nav>
  );
}
```

```css
/* styles/rtl-overrides.css */
/* Specific overrides for complex layouts */
[dir="rtl"] .sidebar {
  border-right: none;
  border-left: 1px solid #e5e7eb;
}

[dir="rtl"] .chevron-icon {
  transform: rotate(180deg);
}

/* Flip asymmetrical images for cultural appropriateness if needed */
[dir="rtl"] .hero-image {
  transform: scaleX(-1);
}
```

## 31.6 SEO for Multi-Language Sites

Optimize search engine visibility across locales with hreflang tags and sitemaps.

### Hreflang Implementation

```typescript
// app/[locale]/sitemap.ts
import { MetadataRoute } from 'next';
import { locales } from '@/config';
import { getAllPosts } from '@/lib/db/queries';

export default async function sitemap(): Promise<MetadataRoute.Sitemap> {
  const posts = await getAllPosts();
  const baseUrl = process.env.NEXT_PUBLIC_URL;
  
  // Generate entries for each locale
  const entries: MetadataRoute.Sitemap = [];
  
  // Home pages
  for (const locale of locales) {
    entries.push({
      url: `${baseUrl}/${locale}`,
      lastModified: new Date(),
      changeFrequency: 'daily',
      priority: 1,
      alternates: {
        languages: Object.fromEntries(
          locales.map(l => [l, `${baseUrl}/${l}`])
        ),
      },
    });
  }
  
  // Blog posts for each locale
  for (const post of posts) {
    for (const locale of locales) {
      entries.push({
        url: `${baseUrl}/${locale}/blog/${post.slug}`,
        lastModified: post.updatedAt,
        changeFrequency: 'weekly',
        priority: 0.8,
        alternates: {
          languages: Object.fromEntries(
            locales.map(l => [
              l, 
              `${baseUrl}/${l}/blog/${post.slug}`
            ])
          ),
        },
      });
    }
  }
  
  return entries;
}

// Metadata generation with hreflang
// app/[locale]/layout.tsx
import { Metadata } from 'next';

export async function generateMetadata({ 
  params: { locale } 
}: { 
  params: { locale: string } 
}): Promise<Metadata> {
  const t = await getTranslations({ locale, namespace: 'metadata' });
  
  return {
    title: {
      template: '%s | Next.js Mastery',
      default: t('title'),
    },
    description: t('description'),
    alternates: {
      canonical: `/${locale}`,
      languages: {
        'en-US': '/en',
        'en-GB': '/en',
        'es-ES': '/es',
        'es-MX': '/es',
        'fr-FR': '/fr',
        'de-DE': '/de',
        'ar-SA': '/ar',
        'ja-JP': '/ja',
      },
    },
    openGraph: {
      locale: locale.replace('-', '_'), // en-US format
      alternateLocale: locales
        .filter(l => l !== locale)
        .map(l => l.replace('-', '_')),
    },
  };
}
```

### Locale-Aware Robots.txt

```typescript
// app/robots.ts
import { MetadataRoute } from 'next';
import { locales } from '@/config';

export default function robots(): MetadataRoute.Robots {
  const baseUrl = process.env.NEXT_PUBLIC_URL;
  
  return {
    rules: {
      userAgent: '*',
      allow: '/',
      disallow: ['/api/', '/_next/'],
    },
    sitemap: locales.map(locale => `${baseUrl}/${locale}/sitemap.xml`),
  };
}
```

## Key Takeaways from Chapter 31

1. **next-intl Configuration**: Use `next-intl` with middleware for locale routing, message loading, and type-safe translations. Configure `localePrefix: 'as-needed'` to hide the default locale from URLs while preserving it for others.

2. **Locale Detection**: Implement middleware-based detection with `localeDetection: true` to automatically redirect based on `Accept-Language` headers or cookies. Always validate locales against a whitelist to prevent 404 errors.

3. **Type Safety**: Define message structures in JSON files and declare them in a TypeScript module to enable autocompletion and compile-time checking of translation keys across your application.

4. **RTL Support**: Use Tailwind's logical properties (e.g., `ms-4` for margin-start instead of `ml-4`) to handle layout direction automatically. Set `dir="rtl"` on the HTML element and load locale-specific fonts for Arabic/Hebrew scripts.

5. **Content Localization**: Store translations either as separate database rows with locale keys or use CMS localization features. Implement fallback chains (e.g., `es-MX` → `es` → `en`) to ensure content availability.

6. **SEO Optimization**: Generate alternate hreflang links for every page using `generateMetadata`, create locale-specific sitemaps, and ensure canonical URLs point to the current locale version to prevent duplicate content penalties.

7. **Formatting**: Use `useFormatter` or `Intl` APIs for dates, numbers, and relative times to automatically adapt to locale conventions (e.g., DD/MM/YYYY vs MM/DD/YYYY, comma vs period for decimals).

## Coming Up Next

**Chapter 32: Accessibility (a11y)**

With your application now accessible to global audiences in their native languages, it's crucial to ensure it's accessible to all users regardless of ability. In Chapter 32, we'll explore semantic HTML patterns, ARIA attributes, keyboard navigation, screen reader optimization, focus management, and automated accessibility testing. You'll learn how to build inclusive Next.js applications that comply with WCAG standards and provide excellent experiences for users with disabilities.